In [1]:
from estnltk import Text

from estnltk.taggers import DateTagger
from pandas import read_csv
from estnltk.taggers import RegexTagger

In [2]:
with open("examples_.txt", "r") as fin:
    lines = fin.readlines()
lines = [line.strip() for line in lines]    

In [3]:
lines

['PSA 03042012 - 0,83ng/ml perearsti poolt .',
 'PSA 2010. 3ng/ml, PSA 2012. 1,53ng/ml . - Bx va',
 'PSA 20105,99 ja 26.01.2012 uuesti .',
 'PSA 2011 oli 0 , 4 nG7ml .',
 'PSA 201222,25ng/ml',
 'PSA 2 aastajooksuldünaamikata , eriuuring',
 ':psa 16,81! ! ! ! ! ,',
 'Happe-aluse tasakaal 6.0 ( 5.0 .. 8.0 )',
 'loli 25 mgx1 ja Monoprili 10 mg Kolesterool 2011a',
 'Kolesterool 1k aastas .',
 'Kõrgenenud kolesterool 2a ( mõõdetud ). Ei pea dieetist kinni',
 'Kontr Verekol 08.12a Per-le juurde .',
 's vas munajuha kasvaja op , günekol 3a tagssi .',
 '08.11.2010 PSA 13.12.2011 7,2ng/ml PSADT on väike .',
 'Rütmihäire tsüklipikkus 330 msek',
 'Loote pikkus : \xa0 3 mm - vastab\xa0 5 nädalat 6 päeva.',
 'Põhjendus: PALAT 10 # ALAT maksanäitaja',
 'ärme vähk 2007 aastast cT3N0M0PSA 59ng/ml .',
 'PSA 8,5( püsib aastaid selles väärtus',
 'S , P-PSA 4.130( <4.100 µg/L )',
 'PSA 5,2.',
 'Kolesterool oli 7,9 mmol/l 0',
 'kolesterool 6.4.',
 'Kolesterool 5,2 mmol/l - esialgu dieet .',
 'SK 3900 g , S

In [4]:
datetagger = DateTagger()

In [5]:
vocabulary = 'regexes.csv'

In [6]:
vocabulary = read_csv(vocabulary, na_filter=False, index_col=False)

In [7]:
# ma tegelt kasutaks ridadel 17, 18, 20 ja 21 + asemel *, aga see annab assertionerrori, kui täägima hakkan :(

vocabulary

,_regex_pattern_,_group_,_priority_,normalized,regex_type,value
0,((K|k)olesteroo?l|KOLESTEROOL|(K|k)olester|Cho...,0,1,,measurement_object,kolesterool
1,((mmoo?l?i?|mm|MMOL|mol)(\s*[-/]\s*(L|l))?|MMO...,0,1,,unit,chol_unit
2,"([0-9])+(\s?[,.]\s?[0-9]+)?",0,1,,number,whatever
3,(psa|Psa|S-PSA|[Pp]rostataspetsiifiline\s*anti...,0,1,,measurement_object,psa
4,(ng/mL|ng/L|mk(ro)?g/[Ll]|ng/\s*ml|ng7ml|mg/ml...,0,1,,unit,psa_unit
5,(a[. ]|aasta|kuu|nädal|[Xx]|kord),0,2,,trash,trashy_trash
6,(((S|s)iinus)?r.tm(iline|ilised)?|[Ff]rekv?(en...,0,1,,measurement_object,pulss
7,(([12][0-9]{2})|([3-9][0-9])),0,1,,number,some_int
8,(((l|x|X|lööki))\s*/?\s*(1\s*)?min(utis)?)|/mi...,0,1,,unit,pulss_unit
9,((([Ss]ünni)|([Kk]eha))?(p|P)ikkus|PIKKUS|pikk...,0,1,,measurement_object,pikkus


In [8]:
test_regex_tagger = RegexTagger(vocabulary=vocabulary,
                                        attributes=['regex_type', 'value'],
                                        conflict_resolving_strategy='MAX',
                                        overlapped=False,
                                        layer_name='type')

In [9]:
tagged_lines = []
for line in lines:
    line = Text(line)
    test_regex_tagger.tag(line)
    datetagger.tag(line)
    tagged_lines.append(line)

In [11]:
for line in tagged_lines:
    print(line.text)
    print(line.regex_type)
    print(line.dates)
    print('--------------')

PSA 03042012 - 0,83ng/ml perearsti poolt .
['measurement_object', 'space', 'number', 'space', 'number', 'unit', 'space', 'space', 'space']
SL[Span(2012, {'date_text': '2012', 'groups': "{'LONGYEAR': '2012'}", 'probability': '0.4', 'extracted_values': None, 'type': 'partial_date'})]
--------------
PSA 2010. 3ng/ml, PSA 2012. 1,53ng/ml . - Bx va
['measurement_object', 'space', 'number', 'unit', 'measurement_object', 'space', 'number', 'number', 'unit', 'space', 'trash', 'space']
SL[Span(10. 3, {'date_text': '10. 3', 'groups': "{'MONTH': '3', 'DAY': '10'}", 'probability': '0.3', 'extracted_values': None, 'type': 'partial_date'}),
Span(12. 1, {'date_text': '12. 1', 'groups': "{'MONTH': '1', 'DAY': '12'}", 'probability': '0.3', 'extracted_values': None, 'type': 'partial_date'})]
--------------
PSA 20105,99 ja 26.01.2012 uuesti .
['measurement_object', 'space', 'number', 'space', 'space', 'number', 'space', 'number', 'space', 'space']
SL[Span(2010, {'date_text': '2010', 'groups': "{'LONGYEAR

GRAMMAR:

(MO = measurement_object, S = space)

MEASUREMENT = MO_S_DATE_S_NUMBER_S_UNIT

MEASUREMENT = MO_S_DATE_S_NUMBER

MEASUREMENT = MO_S_NUMBER_S_UNIT

MEASUREMENT = MO_S_NUMBER

TRASH = MO_S_NUMBER_S_TRASH

TRASH = TRASH_S_NUMBER

TRASH = TRASH_S_MO_S_NUMBER

TRASH = TRASH

TRASH = MO_S_DATE